In [20]:
pip install kaggle

Note: you may need to restart the kernel to use updated packages.


In [22]:
import pandas as pd
import json
import json_lines

from kaggle.api.kaggle_api_extended import KaggleApi

# from keys import kaggle_api

## 1. with Kaggle Api
api =KaggleApi()
api.authenticate()

In [23]:
## CONTENT
#####################
 
# 1. Bali Regency data
# 2. Indo Province Data (transittion to kawal data - march 2021)
# 3. World - Country Data

In [24]:
###  -------------------------------------
### 1. COVID DATA BALI REGENCIES
###  -------------------------------------

## From scrapy spider

In [25]:
## Converting Json-Lines File to pandas DataFrame
##########################################

json_file = r'C:\Users\ansve\Coding\Projects-WebScraping\CovidBali\CovidBali\CovidBali\daily.jl'

##Extract data from First TABLE
###############################################

data_first = []
with open(json_file, 'rb') as f:
    count = 0
    for line in json_lines.reader(f, broken=True):
        count+=1
        if count % 2 == 0: #this is the remainder operator
            for x in range(0, 10):
                data_first.append(line['data'][x])

df_first = pd.DataFrame(data_first)
df_first.head()

Error: JSON line can't be decoded. JSONDecodeError('Extra data: line 1 column 875 (char 874)')


,0,1,2,3,4,5,6,7,8,9,10
0,1,JEMBRANA,-7,70,7,579,1,14,1,663,05 Dec 2020
1,2,TABANAN,-10,434,35,1016,0,47,25,1497,05 Dec 2020
2,3,BADUNG,-9,159,27,2299,0,51,18,2509,05 Dec 2020
3,4,DENPASAR,1,166,26,3705,0,90,27,3961,05 Dec 2020
4,5,GIANYAR,-16,234,31,1681,0,75,15,1990,05 Dec 2020


In [26]:
df_first.tail(30)

,0,1,2,3,4,5,6,7,8,9,10
950,1,JEMBRANA,0,48,1,2032,0,72,1,2152,23 Mar 2021
951,2,TABANAN,14,110,4,3813,1,134,19,4057,23 Mar 2021
952,3,BADUNG,5,282,27,6880,3,184,35,7346,23 Mar 2021
953,4,DENPASAR,10,578,37,11165,3,238,50,11981,23 Mar 2021
954,5,GIANYAR,-1,3,16,4290,0,129,15,4422,23 Mar 2021
955,6,BANGLI,4,96,17,1861,2,77,23,2034,23 Mar 2021
956,7,KLUNGKUNG,3,82,7,1407,1,46,11,1535,23 Mar 2021
957,8,KARANGASEM,-13,53,17,1504,1,81,5,1638,23 Mar 2021
958,9,BULELENG,12,167,10,2512,1,114,23,2793,23 Mar 2021
959,10,KABUPATEN LAINNYA,-1,93,1,353,0,4,0,450,23 Mar 2021


In [27]:
## Column names from https://infocorona.baliprov.go.id/API/pendataan/laporan-harian-02.php = df_first and ...-01.php = df_second

columns_first = [
    'No','Kabupaten / County', 'new_treatment', 'total_treatment', 'new_recovered', 'total_recovered', 'new_deaths', 'total_deaths', 'new_cases', 'total_cases', 'Date']

df_first.columns = columns_first
df_first.head()

,No,Kabupaten / County,new_treatment,total_treatment,new_recovered,total_recovered,new_deaths,total_deaths,new_cases,total_cases,Date
0,1,JEMBRANA,-7,70,7,579,1,14,1,663,05 Dec 2020
1,2,TABANAN,-10,434,35,1016,0,47,25,1497,05 Dec 2020
2,3,BADUNG,-9,159,27,2299,0,51,18,2509,05 Dec 2020
3,4,DENPASAR,1,166,26,3705,0,90,27,3961,05 Dec 2020
4,5,GIANYAR,-16,234,31,1681,0,75,15,1990,05 Dec 2020


In [28]:

##Extract data from SECOND TABLE
###############################################
data_second = []

with open(json_file, 'rb') as f:
    count =0
    for line in json_lines.reader(f, broken=True):
        count+= 1
        if count % 2 != 0:
            for x in range(0, 10):
                data_second.append(line['data'][x])
df_second = pd.DataFrame(data_second)

Error: JSON line can't be decoded. JSONDecodeError('Extra data: line 1 column 875 (char 874)')


In [29]:
df_second.tail()

,0,1,2,3,4,5,6,7,8,9,10,11,12
985,6,BANGLI,0,58,1,22,4,1970,0,0,5,2050,26 Mar 2021
986,7,KLUNGKUNG,0,19,1,51,9,1504,0,0,10,1574,26 Mar 2021
987,8,KARANGASEM,0,23,0,165,2,1458,0,0,2,1646,26 Mar 2021
988,9,BULELENG,0,42,0,24,22,2788,0,0,22,2854,26 Mar 2021
989,10,KABUPATEN LAINNYA,1,7,4,118,6,336,0,0,11,461,26 Mar 2021


In [30]:
# # Second data entry
columnName_second =['No', 'Kabupaten / County', 'new PPLN/PMI', 'total history-of-foreign-travel/ migrant-worker (PPLN/PMI)', 'new domestic-travel-history', 'total domestic-travel-history (PPDN)', 'new_local_transmission', 'total_local_transmission',	'new_other_transmission', 'total_other_transmission', 'new_cases' ,'total_cases', 'Date' ]

df_second.columns = columnName_second


In [31]:
df_second = df_second.drop(['Kabupaten / County','new_cases', 'total_cases'], axis=1)

In [32]:
df_second.tail()

,No,new PPLN/PMI,total history-of-foreign-travel/ migrant-worker (PPLN/PMI),new domestic-travel-history,total domestic-travel-history (PPDN),new_local_transmission,total_local_transmission,new_other_transmission,total_other_transmission,Date
985,6,0,58,1,22,4,1970,0,0,26 Mar 2021
986,7,0,19,1,51,9,1504,0,0,26 Mar 2021
987,8,0,23,0,165,2,1458,0,0,26 Mar 2021
988,9,0,42,0,24,22,2788,0,0,26 Mar 2021
989,10,1,7,4,118,6,336,0,0,26 Mar 2021


In [33]:

## Merge both df
#############################
df_merged = pd.merge(df_first, df_second, how= 'left', left_on=['No', "Date"], right_on=['No', "Date"])

df_bali = df_merged.copy()
df_bali.head()

,No,Kabupaten / County,new_treatment,total_treatment,new_recovered,total_recovered,new_deaths,total_deaths,new_cases,total_cases,Date,new PPLN/PMI,total history-of-foreign-travel/ migrant-worker (PPLN/PMI),new domestic-travel-history,total domestic-travel-history (PPDN),new_local_transmission,total_local_transmission,new_other_transmission,total_other_transmission
0,1,JEMBRANA,-7,70,7,579,1,14,1,663,05 Dec 2020,0,26,0,11,1,626,0,0
1,2,TABANAN,-10,434,35,1016,0,47,25,1497,05 Dec 2020,0,21,2,10,23,1466,0,0
2,3,BADUNG,-9,159,27,2299,0,51,18,2509,05 Dec 2020,0,27,0,4,18,2478,0,0
3,4,DENPASAR,1,166,26,3705,0,90,27,3961,05 Dec 2020,0,48,-1,22,28,3891,0,0
4,5,GIANYAR,-16,234,31,1681,0,75,15,1990,05 Dec 2020,0,29,-1,10,16,1951,0,0


In [34]:
df = df_bali.copy()
df.tail(93)

,No,Kabupaten / County,new_treatment,total_treatment,new_recovered,total_recovered,new_deaths,total_deaths,new_cases,total_cases,Date,new PPLN/PMI,total history-of-foreign-travel/ migrant-worker (PPLN/PMI),new domestic-travel-history,total domestic-travel-history (PPDN),new_local_transmission,total_local_transmission,new_other_transmission,total_other_transmission
887,8,KARANGASEM,1,74,0,1450,0,77,1,1601,16 Mar 2021,0,23,1,145,0,1433,0,0
888,9,BULELENG,-13,138,19,2388,2,109,8,2635,16 Mar 2021,0,41,0,24,8,2570,0,0
889,10,KABUPATEN LAINNYA,2,103,4,320,0,4,6,427,16 Mar 2021,0,6,1,111,5,310,0,0
890,1,JEMBRANA,2,110,1,1926,0,71,3,2107,17 Mar 2021,0,26,0,61,3,2020,0,0
891,2,TABANAN,-7,102,29,3748,1,128,23,3978,17 Mar 2021,0,23,0,142,23,3813,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
975,6,BANGLI,4,84,1,1887,0,79,5,2050,26 Mar 2021,0,58,1,22,4,1970,0,0
976,7,KLUNGKUNG,1,97,9,1431,0,46,10,1574,26 Mar 2021,0,19,1,51,9,1504,0,0
977,8,KARANGASEM,2,53,0,1509,0,84,2,1646,26 Mar 2021,0,23,0,165,2,1458,0,0
978,9,BULELENG,10,201,12,2537,0,116,22,2854,26 Mar 2021,0,42,0,24,22,2788,0,0


In [35]:
# delete duplicate columns and convert dtypes
df = df.loc[:,~df.columns.duplicated()]

## fill NaN in Kabubaten Lain fileds (confirmed cases from outside of bali regency but detected in Bali have no match with reference-data e.g)
################################################
df = df.fillna(0)
df[df.isna().any(axis=1)]

## convert columns
df['No'] = df['No'].astype(int)
df['Name_EN'] = df['Kabupaten / County'].astype(str)
df.replace("KABUPATEN LAIN", "Others", inplace=True)
df['Name_EN'] = df['Name_EN'].str.capitalize()

df.dtypes

No                                                             int32
Kabupaten / County                                            object
new_treatment                                                 object
total_treatment                                               object
new_recovered                                                 object
total_recovered                                               object
new_deaths                                                    object
total_deaths                                                  object
new_cases                                                     object
total_cases                                                   object
Date                                                          object
new PPLN/PMI                                                  object
total history-of-foreign-travel/ migrant-worker (PPLN/PMI)    object
new domestic-travel-history                                   object
total domestic-travel-history (PPD

In [36]:
df.head()

,No,Kabupaten / County,new_treatment,total_treatment,new_recovered,total_recovered,new_deaths,total_deaths,new_cases,total_cases,Date,new PPLN/PMI,total history-of-foreign-travel/ migrant-worker (PPLN/PMI),new domestic-travel-history,total domestic-travel-history (PPDN),new_local_transmission,total_local_transmission,new_other_transmission,total_other_transmission,Name_EN
0,1,JEMBRANA,-7,70,7,579,1,14,1,663,05 Dec 2020,0,26,0,11,1,626,0,0,Jembrana
1,2,TABANAN,-10,434,35,1016,0,47,25,1497,05 Dec 2020,0,21,2,10,23,1466,0,0,Tabanan
2,3,BADUNG,-9,159,27,2299,0,51,18,2509,05 Dec 2020,0,27,0,4,18,2478,0,0,Badung
3,4,DENPASAR,1,166,26,3705,0,90,27,3961,05 Dec 2020,0,48,-1,22,28,3891,0,0,Denpasar
4,5,GIANYAR,-16,234,31,1681,0,75,15,1990,05 Dec 2020,0,29,-1,10,16,1951,0,0,Gianyar


In [37]:
## convert date to datetime 
################################

# from datetime import datetime -> datetime.strptime(date, format)
# df["Date"].apply(lambda x: parse(df['Date']))
# or
# df['Date'] = pd.to_datetime(df['Date'])
## also possible

from dateutil.parser import parse
# check if Nan
# df['Date'].isnull().values

df['Date'] = df['Date'].str.strip()
df['Date'] = df['Date'].astype(str) 
df['Date'] = pd.to_datetime(df.Date, errors='coerce')
df['Date'][0]


Timestamp('2020-12-05 00:00:00')

In [38]:
## Reference Data 
## additional data like population, density etc.

# data cleaning, processing
##########################

# add addtitional data
df_ref = pd.read_excel('Bali_reference_data.xlsx')
df_ref = df_ref.drop(columns=['https://en.wikipedia.org/wiki/Bali#cite_note-BPS2019-2', 'https://sp2010.bps.go.id/index.php/site/tabel?tid=321&wid=0', 'Source'])

## adjust regency names of both data sources before merge
##########################
df_ref['Name_EN'] = df_ref['Name_Indo'].str.capitalize()
df_ref.replace('Kota denpasar', 'Denpasar', inplace=True)
df_ref.replace('totals', 'Bali', inplace=True)
df_ref.replace('Totals', 'Bali', inplace=True)

df_ref.head(15)

,Name_Indo,Name_EN,Capital,area_km2,population_2000,population_2010,population_2015,HDI_2014_estimate,Gouvernor,Nr. of Sub-Districts,Nr of Keluruhan/Villages
0,Bali,Bali,NaN,"5,780.06","3,146,999","3,890,757","4,148,588",0.724 (High),NaN,NaN,NaN
1,Buleleng,Buleleng,Singaraja,"1,364.73",558.181,624.125,645.893,0.691 (Medium),Putu Agus Suradnyana,9.0,19/129
2,Jembrana,Jembrana,Negara,841.80,231.806,261.638,271.423,0.686 (Medium),I Putu Artha,5.0,10/41
3,Karangasem,Karangasem,Amlapura,839.54,360.486,396.487,408.487,0.640 (Medium),I Gusti Ayu Mas Sumatri,8.0,3/75
4,Tabanan,Tabanan,Tabanan,839.30,376.03,420.913,435.753,0.726 (High),Ni Putu Eka Wiryastuti,10.0,-/133
5,Bangli,Bangli,Bangli,490.71,193.776,215.353,222.474,0.657 (Medium),I Made Gianyar,4.0,4/68
6,Badung,Badung,Mangupura,418.52,345.863,543.332,615.148,0.779 (High),I Nyoman Giri Prasta,6.0,16/46
7,Gianyar,Gianyar,Gianyar,368.00,393.155,469.777,494.729,0.742 (High),I Made Agus Mahayastra,7.0,6/64
8,Klungkung,Klungkung,Semarapura,315.00,155.262,170.543,175.573,0.683 (Medium),I Nyoman Suwirta,4.0,6/53
9,Kota Denpasar,Denpasar,Denpasar,127.78,532.44,788.589,879.098,0.816 (Very High),Ida Bagus Rai Dharmawijaya Mantra,4.0,16/27


In [39]:
# merge both df on 'Name_EN'
df_merged = df.merge(df_ref, on="Name_EN", how='left')

In [40]:
df_merged.columns

Index(['No', 'Kabupaten / County', 'new_treatment', 'total_treatment',
       'new_recovered', 'total_recovered', 'new_deaths', 'total_deaths',
       'new_cases', 'total_cases', 'Date', 'new PPLN/PMI',
       'total history-of-foreign-travel/ migrant-worker (PPLN/PMI)',
       'new domestic-travel-history', 'total domestic-travel-history (PPDN)',
       'new_local_transmission', 'total_local_transmission',
       'new_other_transmission', 'total_other_transmission', 'Name_EN',
       'Name_Indo', 'Capital', 'area_km2', 'population_2000',
       'population_2010', 'population_2015', 'HDI_2014_estimate', 'Gouvernor',
       'Nr. of Sub-Districts', 'Nr of Keluruhan/Villages'],
      dtype='object')

In [41]:
# # convert to integers
int_numbers = ['new_treatment', 'total_treatment',
       'new_recovered', 'total_recovered', 'new_deaths', 'total_deaths',
       'new_cases', 'total_cases', 'new PPLN/PMI',
       'total domestic-travel-history (PPDN)',
       'new domestic-travel-history', 'total domestic-travel-history (PPDN)',
       'new_local_transmission', 'total_local_transmission',
       'new_other_transmission', 'total_other_transmission']
## check for duplicated columns
# print(df_merged.columns[df_merged.columns.duplicated(keep=False)])
df_merged[int_numbers] = df_merged[int_numbers].apply(pd.to_numeric)
df_merged.tail()

,No,Kabupaten / County,new_treatment,total_treatment,new_recovered,total_recovered,new_deaths,total_deaths,new_cases,total_cases,...,Name_Indo,Capital,area_km2,population_2000,population_2010,population_2015,HDI_2014_estimate,Gouvernor,Nr. of Sub-Districts,Nr of Keluruhan/Villages
975,6,BANGLI,4,84,1,1887,0,79,5,2050,...,Bangli,Bangli,490.71,193.776,215.353,222.474,0.657 (Medium),I Made Gianyar,4.0,4/68
976,7,KLUNGKUNG,1,97,9,1431,0,46,10,1574,...,Klungkung,Semarapura,315.00,155.262,170.543,175.573,0.683 (Medium),I Nyoman Suwirta,4.0,6/53
977,8,KARANGASEM,2,53,0,1509,0,84,2,1646,...,Karangasem,Amlapura,839.54,360.486,396.487,408.487,0.640 (Medium),I Gusti Ayu Mas Sumatri,8.0,3/75
978,9,BULELENG,10,201,12,2537,0,116,22,2854,...,Buleleng,Singaraja,"1,364.73",558.181,624.125,645.893,0.691 (Medium),Putu Agus Suradnyana,9.0,19/129
979,10,KABUPATEN LAINNYA,11,77,0,380,0,4,11,461,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [42]:
# ## add columns with new calculated data
#########################################

df_merged['total_cases_per_100k'] = df_merged['total_cases'] / df_merged['population_2015'] * 100
df_merged['total_cases_per_100k'] = df_merged['total_cases_per_100k'].apply(pd.to_numeric).round(2)

df_merged['total_deaths_per_100k'] = df_merged['total_deaths'] / df_merged['population_2015'] * 100
df_merged['total_deaths_per_100k'] = df_merged['total_deaths_per_100k'].apply(pd.to_numeric).round(2)

df_merged['cases7'] = df_merged['new_cases'].rolling(window=7, center=False).mean()
df_merged['cases7_per_100k'] = df_merged['cases7'] / df_merged['population_2015'] * 100
df_merged['cases7_per_100k'] = df_merged['cases7_per_100k'].apply(pd.to_numeric).round(2)

df_merged['total_cases_per_100k'] = df_merged['total_cases'] / df_merged['population_2015'] * 100
df_merged['total_cases_per_100k'] = df_merged['total_cases_per_100k'].apply(pd.to_numeric).round(2)

df_merged['deaths7'] = df_merged['new_deaths'].rolling(window=7, center=False).mean()
df_merged['deaths7_per_100k'] = df_merged['total_deaths_per_100k'].rolling(window=7, center=False).mean()
df_merged['deaths7_per_100k'] = df_merged['deaths7_per_100k'].apply(pd.to_numeric).round(2)

df_merged['CFR'] = df_merged['total_deaths'] / df_merged['total_cases'] * 100
df_merged['CFR'] = df_merged['CFR'].round(2)

## Check new created values
##############################
# df_merged[['total_deaths', 'total_cases', 'total_deaths_per_100k', 'population_2015', 'total_cases_per_100k']]

In [43]:
# infection fatality ratio (IFR), which estimates this proportion of deaths among all infected individuals. The second is case fatality ratio (CFR), which estimates this proportion of deaths among identified confirmed cases. 
# see https://www.who.int/news-room/commentaries/detail/estimating-mortality-from-covid-19

## Calculate in App after region selected and df only contains region

df_merged['growth_rate_new_cases'] = df_merged['new_cases'].pct_change(fill_method ='ffill', periods=7)

In [44]:
## add id from geojson file
geojson_bali = 'new_bali_id.geojson'

# open new geojson (with id) file
bali_geo_id = json.load(open(geojson_bali, 'r'))

## get names of regencies
regencies = []
for regency in bali_geo_id['features']:
  regencies.append(regency['properties']['ADM2_EN'])

  # add id column in df with feature.id from geojson
bali_id_map = {}
for regency in bali_geo_id['features']:
  bali_id_map[regency['properties']['ADM2_EN']] = regency['id']
bali_id_map

## for Foreigners or Other Regency add 'id' = 0
df_merged['id'] = df_merged['Name_Indo'].apply(lambda x: bali_id_map[x] if x in bali_id_map else 0)
df_merged.head()

,No,Kabupaten / County,new_treatment,total_treatment,new_recovered,total_recovered,new_deaths,total_deaths,new_cases,total_cases,...,Nr of Keluruhan/Villages,total_cases_per_100k,total_deaths_per_100k,cases7,cases7_per_100k,deaths7,deaths7_per_100k,CFR,growth_rate_new_cases,id
0,1,JEMBRANA,-7,70,7,579,1,14,1,663,...,10/41,244.27,5.16,NaN,NaN,NaN,NaN,2.11,NaN,5
1,2,TABANAN,-10,434,35,1016,0,47,25,1497,...,-/133,343.54,10.79,NaN,NaN,NaN,NaN,3.14,NaN,9
2,3,BADUNG,-9,159,27,2299,0,51,18,2509,...,16/46,407.87,8.29,NaN,NaN,NaN,NaN,2.03,NaN,1
3,4,DENPASAR,1,166,26,3705,0,90,27,3961,...,16/27,450.58,10.24,NaN,NaN,NaN,NaN,2.27,NaN,8
4,5,GIANYAR,-16,234,31,1681,0,75,15,1990,...,6/64,402.24,15.16,NaN,NaN,NaN,NaN,3.77,NaN,4


In [45]:
df_merged.tail()

,No,Kabupaten / County,new_treatment,total_treatment,new_recovered,total_recovered,new_deaths,total_deaths,new_cases,total_cases,...,Nr of Keluruhan/Villages,total_cases_per_100k,total_deaths_per_100k,cases7,cases7_per_100k,deaths7,deaths7_per_100k,CFR,growth_rate_new_cases,id
975,6,BANGLI,4,84,1,1887,0,79,5,2050,...,4/68,921.46,35.51,18.428571,8.28,0.857143,NaN,3.85,-0.772727,2
976,7,KLUNGKUNG,1,97,9,1431,0,46,10,1574,...,6/53,896.49,26.20,19.857143,11.31,0.857143,29.24,2.92,inf,7
977,8,KARANGASEM,2,53,0,1509,0,84,2,1646,...,3/75,402.95,20.56,19.571429,4.79,0.857143,28.39,5.10,-0.500000,6
978,9,BULELENG,10,201,12,2537,0,116,22,2854,...,19/129,441.87,17.96,20.285714,3.14,0.857143,26.40,4.06,0.294118,3
979,10,KABUPATEN LAINNYA,11,77,0,380,0,4,11,461,...,NaN,NaN,NaN,14.428571,NaN,0.571429,NaN,0.87,-0.788462,0


In [46]:
# create csv for Bali Dash App
df_merged.to_csv('bali_regency_data.csv')


In [47]:
# save a version in Kaggle Datasets (upload daily)

df_merged.to_csv(r'C:\Users\ansve\Coding\Kaggle\Datasets\Covid19_Bali\Covid19_Bali_Regencies_Data.csv')

In [27]:
###  -------------------------------------
### 2. INDONESIA COVID DATA 
###  -------------------------------------

## From Kaggel Dataset
# 1. connect to kaggle and get latest dataset


In [28]:
# downoad single file
#Signature: dataset_download_file(dataset, file_name, path=None, force=False, quiet=True)
api.dataset_download_files('hendratno/covid19-indonesia/covid_19_indonesia_time_series_all.csv', unzip = True)
# 

In [29]:
df_indo = pd.read_csv('covid_19_indonesia_time_series_all.csv')

df_indo.tail()

,Date,Location ISO Code,Location,New Cases,New Deaths,New Recovered,New Active Cases,Total Cases,Total Deaths,Total Recovered,...,Longitude,Latitude,New Cases per Million,Total Cases per Million,New Deaths per Million,Total Deaths per Million,Case Fatality Rate,Case Recovered Rate,Growth Factor of New Cases,Growth Factor of New Deaths
10689,1/21/2021,ID-SG,Sulawesi Tenggara,77,1,1,75,9016,172,7392,...,122.070311,-4.124689,29.22,3421.03,0.38,65.26,1.91%,81.99%,1.97,NaN
10690,1/21/2021,ID-SA,Sulawesi Utara,92,2,83,7,11931,361,8497,...,124.521240,1.259638,34.82,4516.10,0.76,136.64,3.03%,71.22%,0.39,1.00
10691,1/21/2021,ID-SB,Sumatera Barat,181,4,49,128,26091,577,18591,...,100.465062,-0.850253,32.79,4727.28,0.72,104.54,2.21%,71.25%,1.08,0.57
10692,1/21/2021,ID-SS,Sumatera Selatan,101,6,49,46,13437,652,10834,...,104.169465,-3.216212,12.29,1635.16,0.73,79.34,4.85%,80.63%,0.92,6.00
10693,1/21/2021,ID-SU,Sumatera Utara,83,1,80,2,19962,719,17228,...,99.051964,2.191894,5.58,1341.99,0.07,48.34,3.60%,86.30%,1.04,0.50


In [30]:
## convert dates to datetime format so matching with new dataframe before merging

print(df_indo['Date'][0])
df_indo['Date'] = pd.to_datetime(df_indo['Date'])


3/1/2020


In [31]:
## Dont Need since transfer to kawalcovid19 Data
## Get Bali and Indo Data from new Xlsx Sheet
## Source Kawal-Indo

# df_indo_new = pd.read_excel('Indo_data.xlsx', 'Bali and Indo')
# df_indo_new= df_indo_new.sort_values(by=['Date'], ascending=True)
# df_indo_new['Date'][0]

# ## Merge with old data df

# df_indo = df_indo.append(df_indo_new)

# # check values
# df_indo.head()


In [32]:
df_indo.columns

Index(['Date', 'Location ISO Code', 'Location', 'New Cases', 'New Deaths',
       'New Recovered', 'New Active Cases', 'Total Cases', 'Total Deaths',
       'Total Recovered', 'Total Active Cases', 'Location Level',
       'City or Regency', 'Province', 'Country', 'Continent', 'Island',
       'Time Zone', 'Special Status', 'Total Regencies', 'Total Cities',
       'Total Districts', 'Total Urban Villages', 'Total Rural Villages',
       'Area (km2)', 'Population', 'Population Density', 'Longitude',
       'Latitude', 'New Cases per Million', 'Total Cases per Million',
       'New Deaths per Million', 'Total Deaths per Million',
       'Case Fatality Rate', 'Case Recovered Rate',
       'Growth Factor of New Cases', 'Growth Factor of New Deaths'],
      dtype='object')

In [33]:
# adjust column names
columns_new = ['Date', 'Location ISO Code', 'Location', 'new_cases', 'new_deaths',
       'new_recovered', 'New Active Cases', 'total_cases', 'total_deaths',
       'total_recovered', 'Total Active Cases', 'Location Level',
       'City or Regency', 'Province', 'Country', 'Continent', 'Island',
       'Time Zone', 'Special Status', 'Total Regencies', 'Total Cities',
       'Total Districts', 'Total Urban Villages', 'Total Rural Villages',
       'Area (km2)', 'Population', 'Population Density', 'Longitude',
       'Latitude', 'New Cases per Million', 'Total Cases per Million',
       'New Deaths per Million', 'Total Deaths per Million',
       'CFR', 'Case Recovered Rate',
       'Growth Factor of New Cases', 'Growth Factor of New Deaths']

df_indo.columns = columns_new
df_indo.columns

Index(['Date', 'Location ISO Code', 'Location', 'new_cases', 'new_deaths',
       'new_recovered', 'New Active Cases', 'total_cases', 'total_deaths',
       'total_recovered', 'Total Active Cases', 'Location Level',
       'City or Regency', 'Province', 'Country', 'Continent', 'Island',
       'Time Zone', 'Special Status', 'Total Regencies', 'Total Cities',
       'Total Districts', 'Total Urban Villages', 'Total Rural Villages',
       'Area (km2)', 'Population', 'Population Density', 'Longitude',
       'Latitude', 'New Cases per Million', 'Total Cases per Million',
       'New Deaths per Million', 'Total Deaths per Million', 'CFR',
       'Case Recovered Rate', 'Growth Factor of New Cases',
       'Growth Factor of New Deaths'],
      dtype='object')

In [34]:
df_indo['Population'].iloc[1]


10846145

In [35]:
# add columns so consistent with Bali_regency data
df_indo['Name_EN'] = df_indo['Location'].str.casefold()

df_indo['new_cases_per_100k'] = df_indo['new_cases'] / df_indo['Population'] * 100000.0
df_indo['new_cases_per_100k'] = df_indo['new_cases_per_100k'].round(2)

df_indo['new_deaths_per_100k'] = df_indo['new_deaths'] / df_indo['Population'] * 100000.0
df_indo['new_deaths_per_100k']= df_indo['new_deaths_per_100k'].round(2)

df_indo['total_cases_per_100k'] = df_indo['total_cases'] / df_indo['Population'] * 100000.0
df_indo['total_cases_per_100k'] = df_indo['total_cases_per_100k'].round(2)

df_indo['total_deaths_per_100k'] = df_indo['total_deaths'] / df_indo['Population'] * 100000.0
df_indo['total_deaths_per_100k'] = df_indo['total_deaths_per_100k'].round(2)

# rolling 7-day average / in App calculated after df only has selected_region 
# df_indo['cases7'] = df_indo['new_cases'].rolling(window=7, center=False).mean().round(2)
# df_indo['deaths7'] = df_indo['new_deaths'].rolling(window=7, center = False).mean().round(2)

# df_indo['cases7_per_100k'] = df_indo['cases7'] / df_indo['Population'] *100000
# df_indo['deaths7_per_100k'] = df_indo['deaths7'] / df_indo['Population'] *100000
# df_indo['cases7_per_100k'] = df_indo['cases7_per_100k'].round(2)
# df_indo['deaths7_per_100k'] = df_indo['deaths7_per_100k'].round(2)

# df_indo['growth_rate'] = df_indo['Growth Factor of New Cases']
# df_indo['growth_rate_new_cases'] = df_indo['new_cases'].pct_change(periods=7, fill_method='ffill')

# CFR
df_indo['CFR'] = df_indo['total_deaths'] / df_indo['total_cases'] * 100


In [36]:
## add id from geojson for choropleth Map
#########################################

# open new geojson (with id) file
geojson_indo = 'new_indo_id.geojson'
indo_geo_id = json.load(open(geojson_indo, 'r'))


In [37]:
indo_geo_id['features'][7]['id']

8

In [38]:
## ADJUST NAMES OF df_indo with NAMES OF GEOJSON ADM1_EN

# which names are different
names_df = df_indo['Location'].unique()
provinces = []
for province in indo_geo_id['features']:
  provinces.append(province['properties']['state'])

list(set(provinces) - set(names_df))

['Bangka-Belitung', 'Irian Jaya Barat', 'Yogyakarta', 'Jakarta Raya']

In [39]:
list_df = ['DKI Jakarta', 'Kalimantan Utara', 'Daerah Istimewa Yogyakarta', 'Papua Barat','Kepulauan Bangka Belitung']

list(set(provinces) - set(names_df))
list1= ['Bangka-Belitung', 'Jakarta Raya', 'Yogyakarta', 'Irian Jaya Barat']


In [40]:
# adjust Names of provnices in Indo

df_indo = df_indo.replace("DKI Jakarta", 'Jakarta Raya')
df_indo = df_indo.replace("Papua Barat", 'Irian Jaya Barat')
df_indo = df_indo.replace("Daerah Istimewa Yogyakarta", 'Yogyakarta')
df_indo = df_indo.replace("Kepulauan Bangka Belitung", 'Bangka-Belitung')

In [41]:
provinces = []
for province in indo_geo_id['features']:
  provinces.append(province['properties']['state'])

# add id column in df with feature.id from geojson
indo_id_map = {}
for province in indo_geo_id['features']:
  indo_id_map[province['properties']['state']] = province['id']
# indo_id_map

# ## for Foreigners or Other Regency add 'id' = 0
df_indo['id'] = df_indo['Location'].apply(lambda x: indo_id_map[x] if x in indo_id_map else 0)

df_indo.tail(100)


,Date,Location ISO Code,Location,new_cases,new_deaths,new_recovered,New Active Cases,total_cases,total_deaths,total_recovered,...,CFR,Case Recovered Rate,Growth Factor of New Cases,Growth Factor of New Deaths,Name_EN,new_cases_per_100k,new_deaths_per_100k,total_cases_per_100k,total_deaths_per_100k,id
10594,2021-01-19,ID-YO,Yogyakarta,287,1,107,179,17515,395,11761,...,2.255210,67.15%,0.97,0.17,daerah istimewa yogyakarta,7.90,0.03,482.37,10.88,3
10595,2021-01-19,ID-GO,Gorontalo,28,0,27,1,4108,107,3868,...,2.604674,94.16%,4.67,0.00,gorontalo,2.37,0.00,347.94,9.06,26
10596,2021-01-19,IDN,Indonesia,10365,308,8013,2044,927380,26590,753948,...,2.867217,81.30%,1.14,1.04,indonesia,3.91,0.12,349.71,10.03,0
10597,2021-01-19,ID-JA,Jambi,49,0,5,44,4024,61,2760,...,1.515905,68.59%,1.69,1.00,jambi,1.40,0.00,115.19,1.75,28
10598,2021-01-19,ID-JB,Jawa Barat,1684,24,1362,298,115756,1399,94133,...,1.208577,81.32%,1.13,0.86,jawa barat,3.73,0.05,256.32,3.10,29
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10689,2021-01-21,ID-SG,Sulawesi Tenggara,77,1,1,75,9016,172,7392,...,1.907720,81.99%,1.97,NaN,sulawesi tenggara,2.92,0.04,342.10,6.53,23
10690,2021-01-21,ID-SA,Sulawesi Utara,92,2,83,7,11931,361,8497,...,3.025731,71.22%,0.39,1.00,sulawesi utara,3.48,0.08,451.61,13.66,22
10691,2021-01-21,ID-SB,Sumatera Barat,181,4,49,128,26091,577,18591,...,2.211491,71.25%,1.08,0.57,sumatera barat,3.28,0.07,472.73,10.45,2
10692,2021-01-21,ID-SS,Sumatera Selatan,101,6,49,46,13437,652,10834,...,4.852274,80.63%,0.92,6.00,sumatera selatan,1.23,0.07,163.52,7.93,33


In [42]:
df_indo.to_csv('indo_province_data.csv')

In [43]:
### 3. World Data? Comparison Data
##------------------------

# https://github.com/owid/covid-19-data/blob/master/public/data/owid-covid-data.csv

url = "https://raw.githubusercontent.com/owid/covid-19-data/master/public/data/owid-covid-data.csv"
df_world = pd.read_csv(url)

In [44]:
df_world.head()

,iso_code,continent,location,date,total_cases,new_cases,new_cases_smoothed,total_deaths,new_deaths,new_deaths_smoothed,...,gdp_per_capita,extreme_poverty,cardiovasc_death_rate,diabetes_prevalence,female_smokers,male_smokers,handwashing_facilities,hospital_beds_per_thousand,life_expectancy,human_development_index
0,AFG,Asia,Afghanistan,2020-02-24,1.0,1.0,NaN,NaN,NaN,NaN,...,1803.987,NaN,597.029,9.59,NaN,NaN,37.746,0.5,64.83,0.511
1,AFG,Asia,Afghanistan,2020-02-25,1.0,0.0,NaN,NaN,NaN,NaN,...,1803.987,NaN,597.029,9.59,NaN,NaN,37.746,0.5,64.83,0.511
2,AFG,Asia,Afghanistan,2020-02-26,1.0,0.0,NaN,NaN,NaN,NaN,...,1803.987,NaN,597.029,9.59,NaN,NaN,37.746,0.5,64.83,0.511
3,AFG,Asia,Afghanistan,2020-02-27,1.0,0.0,NaN,NaN,NaN,NaN,...,1803.987,NaN,597.029,9.59,NaN,NaN,37.746,0.5,64.83,0.511
4,AFG,Asia,Afghanistan,2020-02-28,1.0,0.0,NaN,NaN,NaN,NaN,...,1803.987,NaN,597.029,9.59,NaN,NaN,37.746,0.5,64.83,0.511


In [45]:
pd.unique(df_world['location'])

array(['Afghanistan', 'Africa', 'Albania', 'Algeria', 'Andorra', 'Angola',
       'Anguilla', 'Antigua and Barbuda', 'Argentina', 'Armenia', 'Asia',
       'Australia', 'Austria', 'Azerbaijan', 'Bahamas', 'Bahrain',
       'Bangladesh', 'Barbados', 'Belarus', 'Belgium', 'Belize', 'Benin',
       'Bermuda', 'Bhutan', 'Bolivia', 'Bosnia and Herzegovina',
       'Botswana', 'Brazil', 'Brunei', 'Bulgaria', 'Burkina Faso',
       'Burundi', 'Cambodia', 'Cameroon', 'Canada', 'Cape Verde',
       'Cayman Islands', 'Central African Republic', 'Chad', 'Chile',
       'China', 'Colombia', 'Comoros', 'Congo', 'Costa Rica',
       "Cote d'Ivoire", 'Croatia', 'Cuba', 'Cyprus', 'Czechia',
       'Democratic Republic of Congo', 'Denmark', 'Djibouti', 'Dominica',
       'Dominican Republic', 'Ecuador', 'Egypt', 'El Salvador',
       'Equatorial Guinea', 'Eritrea', 'Estonia', 'Eswatini', 'Ethiopia',
       'Europe', 'European Union', 'Faeroe Islands', 'Falkland Islands',
       'Fiji', 'Finland', 'Fran

In [46]:
df_world.columns

Index(['iso_code', 'continent', 'location', 'date', 'total_cases', 'new_cases',
       'new_cases_smoothed', 'total_deaths', 'new_deaths',
       'new_deaths_smoothed', 'total_cases_per_million',
       'new_cases_per_million', 'new_cases_smoothed_per_million',
       'total_deaths_per_million', 'new_deaths_per_million',
       'new_deaths_smoothed_per_million', 'reproduction_rate', 'icu_patients',
       'icu_patients_per_million', 'hosp_patients',
       'hosp_patients_per_million', 'weekly_icu_admissions',
       'weekly_icu_admissions_per_million', 'weekly_hosp_admissions',
       'weekly_hosp_admissions_per_million', 'new_tests', 'total_tests',
       'total_tests_per_thousand', 'new_tests_per_thousand',
       'new_tests_smoothed', 'new_tests_smoothed_per_thousand',
       'positive_rate', 'tests_per_case', 'tests_units', 'total_vaccinations',
       'people_vaccinated', 'people_fully_vaccinated', 'new_vaccinations',
       'new_vaccinations_smoothed', 'total_vaccinations_per_hun

In [47]:
# df_world['location'].unique()

In [48]:
# relevant countries :
rel_countries = ["Germany", 'Europe', 'World', 'International', 'Indonesia',  'United Kingdom', 'Australia', 'France', "Italy", 'United States']

In [49]:
# CFR, total cases per 100k and total deaths, growth_rate, vaccination
## add columns for total cases/deaths per 100k
df_world['Date'] = df_world['date'] 
df_world['CFR'] = df_world['total_deaths'] / df_world['total_cases'] * 100
df_world['total_cases_per_100k'] = df_world['total_cases_per_million'] / 10
df_world['total_deaths_per_100k'] = df_world['total_deaths_per_million'] / 10


In [50]:
df_germany = df_world[df_world['location'] == 'Germany']
df_germany.head()
df_germany['total_cases_per_100k'].iloc[-1]
# df_germany['total_cases_per_million'].iloc[-1]

3250.0117

In [51]:
df_world.to_csv("world_data.csv")